In [ ]:
#This function will scrape results data from the web and save it to a csv file.

def scrape_web_data():

    import requests
    from bs4 import BeautifulSoup
    import html5lib
    import lxml
    import urllib.request
    from selenium import webdriver
    import time
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC

    #list of urls to scrape
    url_list_df = pd.read_excel('race_list.xlsx')
    url_list = url_list_df['URL'].to_list()
    date_list = url_list_df['Date'].to_list()
    race_dfs = pd.DataFrame()

        
    for link, date in zip(url_list, date_list):
        driver = webdriver.Safari()


        stripped_link = link.replace('worldcup.eliotjackson.com/results/', '')
        split_link = stripped_link.split('/')
        year = split_link[0]
        venue = split_link[1]
    
        url = 'http://' + link
        print(url)

        driver.get(url)
    
        #Allow driver to load the entire webpage
        time.sleep(5)


        res = driver.execute_script("return document.documentElement.outerHTML")

        driver.quit()

        soup = BeautifulSoup(res, 'lxml')
        data = []
        table = soup.find('div', attrs={'class':'react-bs-container-body'})
        table_body = table.find('tbody')
    
        #Create a table from the web page's source code
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values
    
        #Filter out relevant data from table, position and rider name
        newdata = data[0::2]
        pos_list = []
        name_list = []
        for item in newdata:
            pos_list.append(item[0])
            name_list.append(item[2])

        race_results = list(zip(pos_list, name_list))
        race_df = pd.DataFrame(race_results, columns=['Rank', 'Name'])
        race_df['Year'] = year
        race_df['Venue'] = venue
        race_df['Date'] = date
        race_df['FirstName'], race_df['LastName'] = race_df['Name'].str.split(' ', 1).str
        race_df['Name'] = race_df.FirstName + ' ' + race_df.LastName.str.upper()
        race_df.drop(['FirstName', 'LastName'], axis=1, inplace=True)
    
        race_dfs = race_dfs.append(race_df, sort=False)
    
    print(race_dfs)
    race_dfs.to_excel("race_dfs_output.xlsx", engine='xlsxwriter')
    return

